In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.stats as stats

mpl.rc('font', family='Malgun Gothic')

In [2]:
df1 = pd.read_csv('01_Data.csv')

In [3]:
df1['Overdue_Type'].value_counts()

없음    49110
있음     2191
Name: Overdue_Type, dtype: int64

In [4]:
# 1. 데이터 전처리 
# 결측치 처리 (dropna : 행 내 결측값이 하나라도 존재하면 해당 행을 삭제)
df1_clean = df1.dropna()

In [5]:
# 2. X와 Y를 선택
X = df1_clean[['Term', 'Amount_Month', 'Credit_Rank', 'Age']]
Y = df1_clean['Overdue_Type'] 

In [6]:
# 3. 학습데이터와 검증데이터 분할 라이브러리 호출
from sklearn.model_selection import train_test_split

In [7]:
# test_size : 테스트 데이터의 사이즈 조정 (defalt = 0.2)
# random_state : 호출할 때마다 동일한 학습/테스트용 데이터를 생성하기 위해 주어지는 난수 값 
#                (아무거나 써도 됨)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, 
                                                    random_state=1234)

In [8]:
# 4. 학습 실시
from sklearn.tree import DecisionTreeClassifier

In [9]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

DecisionTreeClassifier()

In [10]:
# 5. 평가
from sklearn.metrics import classification_report  # 분류모델의 성능 측정 함수

In [11]:
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

In [12]:
# 학습 성능
print(classification_report(Y_train, Y_train_pred))

              precision    recall  f1-score   support

          없음       0.97      1.00      0.99     27423
          있음       0.94      0.27      0.42      1029

    accuracy                           0.97     28452
   macro avg       0.96      0.63      0.70     28452
weighted avg       0.97      0.97      0.97     28452



> 학습 시 train set에 대해 정확도가 100가 나와버리면 Overfitting 에 대한 문제가 발생한다.  
> 그렇게 된다면 test set에 대한 정확도가 현저히 낮아진다.

In [13]:
# 일반화 성능
print(classification_report(Y_test, Y_test_pred))

              precision    recall  f1-score   support

          없음       0.97      0.99      0.98     11795
          있음       0.24      0.10      0.14       400

    accuracy                           0.96     12195
   macro avg       0.60      0.54      0.56     12195
weighted avg       0.95      0.96      0.95     12195



# Pipe Line  파이프라인

In [14]:
!pip install imblearn --user

In [15]:
from imblearn.pipeline import make_pipeline  # 특성 공학 + 학습 파이프 생성함수
from sklearn.preprocessing import MinMaxScaler  # 숫자 스케일링 (0~1)
from sklearn.preprocessing import OneHotEncoder  # 문자 인코딩 (항목 -> 1 | 0)
# 각 항목의 타입별로 파이프 분할
from sklearn.compose import make_column_transformer
# 결측치 대치  
from sklearn.impute import SimpleImputer # 단순 대치

In [16]:
# 숫자 데이터 (결측값을 평균으로 처리 -> 스케일링)
numeric_pipe = make_pipeline((SimpleImputer(strategy='mean')), 
                             (MinMaxScaler()))
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('minmaxscaler', MinMaxScaler())])

In [17]:
# 문자데이터 (결측값을 최빈값으로 처리 -> 인코딩)
category_pipe = make_pipeline((SimpleImputer(strategy='most_frequent')), 
                              (OneHotEncoder()))
category_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder())])

In [18]:
X = df1[['Term', 'Amount_Month', 'Age', 'Credit_Rank', 'Gender', 'Product_Type']]
Y = df1['Overdue_Type']

In [19]:
X.describe().columns.tolist() # 숫자 항목만 추출

['Term', 'Amount_Month', 'Age', 'Credit_Rank']

In [20]:
X.describe(include='object').columns.tolist() # 문자 항목만 추출

['Gender', 'Product_Type']

In [21]:
# 숫자 항목은 숫자끼리 문자 항목은 문자끼리 분류하여 처리
numeric_list = X.describe().columns.tolist()
category_list = X.describe(include='object').columns.tolist()
preprocessing_pipe = make_column_transformer((numeric_pipe, numeric_list), 
                                             (category_pipe, category_list))
preprocessing_pipe

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ['Term', 'Amount_Month', 'Age',
                                  'Credit_Rank']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['Gender', 'Product_Type'])])

In [22]:
# 전처리 파이프 + 학습 알고리즘
model_pipe = make_pipeline(preprocessing_pipe, DecisionTreeClassifier())
model_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['Term', 'Amount_Month',
                                                   'Age', 'Credit_Rank']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['Gender',
                                                   'Product_Type'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [23]:
model_pipe.fit(X, Y)  # 학습 수행

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['Term', 'Amount_Month',
                                                   'Age', 'Credit_Rank']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['Gender',
                                                   'Product_Type'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [24]:
# 학습 모델 파일형태 저장
import pickle  # 파이썬에서 선언된 객체를 파일로 변환

In [25]:
pickle.dump(model_pipe, open('model.sav', 'wb'))  # model.sav 저장